<a href="https://colab.research.google.com/github/Bellecho/24point/blob/main/music_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === Colab Cell 1: 环境准备 ===
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip -q install librosa==0.10.2.post1 soundfile pretty_midi miditoolkit numpy pandas scikit-learn matplotlib einops tqdm

import os, sys, math, json, random, glob
import numpy as np
import pandas as pd
import librosa, librosa.display, soundfile as sf
import pretty_midi, miditoolkit
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from einops import rearrange
SEED = 2025
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.9 MB/s eta 0:00:00


In [2]:
# 1) 克隆仓库
!git clone https://github.com/lucasnfe/vgmidi.git
!ls -l vgmidi | head

# 2) 看看有啥文件（应该能看到 labelled/、unlabelled/、vgmidi_labelled.csv 等）
!ls -l vgmidi/labelled | head

# 3) 用 pandas 打开标注表，确认列名
import pandas as pd, os
df = pd.read_csv("/content/vgmidi/vgmidi_labelled.csv")
print(df.columns.tolist())
df.head(3)


Cloning into 'vgmidi'...
remote: Enumerating objects: 23026, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 23026 (delta 0), reused 1 (delta 0), pack-reused 23023 (from 1)
Receiving objects: 100% (23026/23026), 1.44 GiB | 25.18 MiB/s, done.
Resolving deltas: 100% (5490/5490), done.
Updating files: 100% (400/400), done.
total 996
drwxr-xr-x 6 root root   4096 Sep 29 05:04 labelled
-rw-r--r-- 1 root root   1085 Sep 29 05:03 README.md
drwxr-xr-x 5 root root   4096 Sep 29 05:04 unlabelled
-rw-r--r-- 1 root root  27908 Sep 29 05:04 vgmidi_labelled.csv
-rw-r--r-- 1 root root 975776 Sep 29 05:04 vgmidi_unlabelled.csv
total 29272
drwxr-xr-x 2 root root     4096 Sep 29 05:03 annotations
drwxr-xr-x 2 root root    12288 Sep 29 05:04 audio
drwxr-xr-x 2 root root    20480 Sep 29 05:04 midi
-rw-r--r-- 1 root root   406041 Sep 29 05:04 phrases.zip
-rw-r--r-- 1 root root 29520837 Sep 29 05:04 plots.zip
drwxr-xr-x 2 root root     4096 Sep 

,id,series,console,game,piece,midi,valence,arousal
0,8013,Banjo-Kazooie,N64,Banjo-Kazooie,Boggys Igloo Happy,labelled/phrases/Banjo-Kazooie_N64_Banjo-Kazoo...,1,1
1,8073,Banjo-Kazooie,N64,Banjo-Kazooie,Boggys Igloo Sad,labelled/phrases/Banjo-Kazooie_N64_Banjo-Kazoo...,1,-1
2,8029,Banjo-Kazooie,N64,Banjo-Kazooie,Bubblegloop Swamp,labelled/phrases/Banjo-Kazooie_N64_Banjo-Kazoo...,1,1


In [6]:
import numpy as np, glob, os, pandas as pd

# 1) 读取元表
raw = pd.read_csv("/content/vgmidi/vgmidi_labelled.csv")

# 2) 猜测/选择文件名列（包含 .mid 的那列）
midi_col_candidates = [c for c in raw.columns if raw[c].astype(str).str.endswith(".mid").any()]
assert len(midi_col_candidates)>=1, "找不到包含 .mid 的列，请手工查看 df.head() 并把列名填到 midi_col"
midi_col = midi_col_candidates[0]

# 3) 找到 valence / arousal 两列（不确定具体列名时，模糊匹配）
v_cols = [c for c in raw.columns if "valence" in c.lower()]
a_cols = [c for c in raw.columns if "arousal" in c.lower()]
assert len(v_cols)>=1 and len(a_cols)>=1, "标注表里找不到 valence/arousal 字样的列，请手工确认列名"
v = raw[v_cols[0]].astype(float)
a = raw[a_cols[0]].astype(float)

# 4) 归一化到 [0,1] 再用 0.5 做阈值（数据量纲不确定时最稳妥）
v01 = (v - v.min())/(v.max()-v.min() + 1e-9)
a01 = (a - a.min())/(a.max()-a.min() + 1e-9)

def to_quadrant(v, a):
    # 高唤醒/高效价：Q1；高唤醒/低效价：Q2；低唤醒/低效价：Q3；低唤醒/高效价：Q4
    hi_v, hi_a = v>0.5, a>0.5
    if hi_a and hi_v: return "Q1"
    if hi_a and not hi_v: return "Q2"
    if (not hi_a) and (not hi_v): return "Q3"
    return "Q4"

label = [to_quadrant(v01[i], a01[i]) for i in range(len(v01))]

# 5) 组成 midi 的绝对路径
raw['midi_path'] = raw[midi_col].apply(lambda s: os.path.join("/content/vgmidi/labelled", os.path.basename(str(s))))

# 6) 生成 meta（暂时 audio_path 置空，等你在 C 部分渲染 WAV）
meta = pd.DataFrame({
    "id": np.arange(len(raw)).astype(str).str.zfill(4),
    "audio_path": "",        # 先空着；如果只做 MIDI 分支可一直为空
    "midi_path": raw['midi_path'],
    "label": label
})
os.makedirs("/content/data/vgmidi", exist_ok=True)
meta.to_csv("/content/data/vgmidi/meta.csv", index=False)
meta.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/vgmidi/vgmidi_labelled.csv'